In [2]:
from typing import Tuple

import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()
from tensorflow import keras

from constants import TRAINING_PARTITIONS, ALL_PARTITIONS, annotation_mapping, amino_acid_mapping, reverse_annotation_mapping
from constants import TYPES, KINGDOMS, METRIC_KINGDOMS, METRIC_TYPES
from metrics.metrics import *
from utils.Dataset import Dataset
from utils.helpers import getDatasetPath
from utils.encoding import categoricalToSequence, oneHotToCategorical, sequenceToCategorical, categoricalToOneHot
from serialization import Serializer

# Load

In [3]:
# Load model
run_timestamp = "20211113-1438"
base_path = f"../results/{run_timestamp}/"
final_model = keras.models.load_model(base_path + f"models/final_model.h5")

# Load logged metrics
final_training_metrics = pd.read_csv(
        base_path + f"metrics/final_model.csv",
        index_col="epoch"
)

# 1-based indexing for epochs
final_training_metrics.reset_index(inplace=True)
final_training_metrics["epoch"] = final_training_metrics["epoch"] + 1
final_training_metrics.set_index(["epoch"], inplace=True)

# Evaluation on Test Set

## Load test set

In [4]:
test_data = Dataset(getDatasetPath()).getFolds([0])

## Make predictions

In [5]:
test_x = np.array([categoricalToOneHot(sequenceToCategorical(seq, amino_acid_mapping), amino_acid_mapping) for seq in test_data["sequence"]])
predictions = final_model.predict(test_x)

test_data["prediction"] = np.array([categoricalToSequence(oneHotToCategorical(pred), reverse_annotation_mapping) for pred in predictions])

In [19]:
test_data

sequence  \
partition type  kingdom  number                                                      
0         SP    EUKARYA  0       MKAVITLLFLACILVVTYGDLICGTNYCKDHPCTSPIARASCRSPA...   
                         1       MVRPKHQPGGLCLLLLLLCQFMEDRSAQAGNCWLRQAKNGRCQVLY...   
                         2       MNSVLFLTLAVCSSLAYGKEFVATVRQNYKENINQLLEQQIQKELA...   
                         3       MVRARHQPGGLCLLLLLLCQFMEDRSAQAGNCWLRQAKNGRCQVLY...   
                         4       MRLLLALLGVLLSVPGPPVLSLEASEEVELEPCLAPSLEQQEQELT...   
...                                                                            ...   
          NO_SP NEGATIVE 79      MLNQKIQNPNPDELMIEVDLCYELDPYELKLDEMIEAEPEPEMIEG...   
                         80      MKKISRKEYVSMYGPTTGDKVRLGDTDLIAEVEHDYTIYGEELKFG...   
                         81      MTPLVKDIIMSSTRMPALFLGHGSPMNVLEDNLYTRSWQKLGMTLP...   
                         82      MRIKPDDNWRWYYDEEHDRMMLDLANGMLFRSRFARKMLTPDAFSP...   
                         83      MQSVTPTSQYLKALNEGSHQPDDVQKEAVSRLEIIYQELINSTPPA...   

                                                                        annotation  \
partition type  kingdom  number                                                      
0         SP    EUKARYA  0       SSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                         1       SSSSSSSSSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOO...   
                         2       SSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                         3       SSSSSSSSSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOO...   
                         4       SSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOO...   
...                                                                            ...   
          NO_SP NEGATIVE 79      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...   
                         80      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...   
                         81      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...   
                         82      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...   
                         83      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...   

                                                                        prediction  
partition type  kingdom  number                                                     
0         SP    EUKARYA  0       SSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOO...  
                         1       SSSSSSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOO...  
                         2       SSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                         3       SSSSSSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOO...  
                         4       SSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOO...  
...                                                                            ...  
          NO_SP NEGATIVE 79      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...  
                         80      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...  
                         81      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...  
                         82      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...  
                         83      IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...  

[4104 rows x 3 columns]

In [9]:
test_data.to_csv("G:\\My Drive\\Files\\Projects\\University\\2021S\\PBL\\Code\\evaluation\\data\\test_split_predictions_julius.tsv", sep="\t")

## Compute stratified metrics

In [15]:
def getRelevantData(query: str) -> Tuple[np.ndarray, np.ndarray]:
    relevant_data = test_data.query(query)
    y_pred = np.array([sequenceToCategorical(seq, annotation_mapping) for seq in relevant_data["prediction"]])
    y_true = np.array([sequenceToCategorical(seq, annotation_mapping) for seq in relevant_data["annotation"]])

    return (y_pred, y_true)


In [26]:
metrics = [MCC(), Recall("macro"), Precision("macro"), Accuracy()]
final_metrics_dict = {}
for metric in metrics:
    # All sequences
    y_pred, y_true = getRelevantData("index == index")
    final_metrics_dict[metric.name] = {
        "overall": {
            "overall": metric(y_true, y_pred)
        }
    }

    # By kingdom
    for kingdom in KINGDOMS:
        y_pred, y_true = getRelevantData(f"kingdom == '{kingdom}'")
        final_metrics_dict[metric.name]["overall"][kingdom] = metric(y_true, y_pred)

    # By pathway
    for type in TYPES:
        y_pred, y_true = getRelevantData(f"type == '{type}'")
        final_metrics_dict[metric.name][type] = {"overall": metric(y_true, y_pred)}

    # By pathway & type
    for type in TYPES:
        for kingdom in KINGDOMS:
            y_pred, y_true = getRelevantData(f"kingdom == '{kingdom}' and type == '{type}'")
            final_metrics_dict[metric.name][type][kingdom] = metric(y_true, y_pred)

# Manually exclude non-existing labels (L & T) from eukarya
y_pred, y_true = getRelevantData("kingdom == 'EUKARYA'")
final_metrics_dict["precision"]["overall"]["EUKARYA"] = precision_score(y_true.flatten(), y_pred.flatten(), average="macro", labels=[0,3,4,5])
final_metrics_dict["recall"]["overall"]["EUKARYA"] = recall_score(y_true.flatten(), y_pred.flatten(), average="macro", labels=[0,3,4,5])

# Convert to dataframe
final_metrics = pd.DataFrame([
    (metric.name, type, kingdom, final_metrics_dict[metric.name][type][kingdom])
    for metric in metrics
    for type in METRIC_TYPES
    for kingdom in METRIC_KINGDOMS
])

final_metrics.columns = ["metric", "type", "kingdom", "value"]
final_metrics.set_index(["metric", "type", "kingdom"])

C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Pr

value
metric   type kingdom           
mcc      LIPO EUKARYA   0.000000
              ARCHAEA   0.795843
              POSITIVE  0.892144
              NEGATIVE  0.898425
              overall   0.894370
...                          ...
accuracy SP   EUKARYA   0.939193
              ARCHAEA   0.833333
              POSITIVE  0.831274
              NEGATIVE  0.875248
              overall   0.921243

[100 rows x 1 columns]

# Store results

In [27]:
Serializer.save(final_metrics, "final_metrics")
Serializer.save(final_training_metrics, "final_training_metrics")